## National Water Model: Time series extraction
Extract an hourly time series from one of the 2.7 million rivers in the 26-year NWM reanalysis

***
Inspired by [Rich Signell's notebook](https://nbviewer.jupyter.org/gist/rsignell-usgs/331eaa1fc9955d9c0e68f6b3ea8c22fb).
***


 - Created by:   __Andy Carter, PE__
 >andy.carter@austin.utexas.edu

 - Created On:   __08 Jan 2021__<br><br>
 - Last revised:  __Verison 0.1__
 
 - Purpose:
 >From the NOAA 'ZARR' repository (on AWS) of the National Water Model v2.0 reanalysis, the goal is to slice a 
 single NetCDF file representing flow for 1 hour timesteps from 1993-01-01 to the last recorded data.  This is about
 26 years.  Attempting to write out one NetCDF file that has the hydrograph for a single COMID.

 - Inputs required:
  > 1) COMID for the requested stream <br>
    2) Path to write out the netCDF<br>
    3) ESRI Shapefile of the area to be sampled (EPSG: zone does not matter)<br>
    
 - Output generated:
  > Single netCDF file of the flow every hour from the NWM v2.0 reanalysis
    

***
### 1.0 References

In [ ]:
import xarray as xr
import fsspec
import hvplot.xarray

***
### 2.0 Input 

In [ ]:
intCOMID = 5790110
strPathHeader = r"C:\NWM_Slice\RiverFlow_"
strPathHeater = "."

In [ ]:
url = r'http://noaa-nwm-retro-v2.0-zarr-pds.s3.amazonaws.com/'

In [ ]:
#Alternate if have read access to NOAA AWS s3 bucket... 

#url = 's3://esip-qhub/noaa/nwm'

***
### 3.0 Fetch Data

In [ ]:
%%time
ds = xr.open_zarr(fsspec.get_mapper(url), consolidated=True)

#Alternate with AWS S3 access credentials
#ds = xr.open_zarr(fsspec.get_mapper(url, requester_pays=True), consolidated=True)

In [ ]:
%%time
dm = ds.streamflow.sel(feature_id=intCOMID, time=slice('2014-09-01','2014-09-30')).load()

In [ ]:
dm.hvplot(grid=True)

In [ ]:
dm.to_series().to_csv('dm.csv')

In [ ]:
from dask.distributed import Client

In [ ]:
client = Client()

In [ ]:
client

In [ ]:
url = 's3://noaa-nwm-retro-v2.0-zarr-pds'

In [ ]:
ds = xr.open_zarr(fsspec.get_mapper(url, anon=True), consolidated=True)

In [ ]:
ds.streamflow

In [ ]:
%%time
dm = ds.streamflow.sel(feature_id=intCOMID).load()

In [ ]:
client.close()

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client
cluster = gateway.new_cluster(environment='pangeo', profile='Pangeo Worker')


In [ ]:
cluster

In [ ]:
cluster.scale(8)

In [ ]:
client = Client(cluster)

In [ ]:
%%time
dm = ds.streamflow.sel(feature_id=intCOMID).load()

***
### 4.0 Save hydrograph data to netCDF on local drive

In [ ]:
strPath = strPathHeader + str(intCOMID) + ".nc" 
dm.to_netcdf(path=strPath, mode="w")

In [ ]:
ds2 = xr.open_dataset(strPath)

In [ ]:
ds2

In [ ]:
ds2['streamflow'].plot(figsize=(12,4))

In [ ]:
ds2['streamflow'].hvplot(grid=True)

***
### 5.0 Notes

-2021.01.08 - It appears that the request to the ZARR via the http request is not returning the full time series data.  Is this
beacuse the local machine does not have aqequate memory for the xarray?  Would a container on an HPC with adequate memory be able to get these data.  Perhaps the ZARR via the HTTP is not complete (not seeing all chunks).  Currently unkown.